In [1]:
import json
import numpy as np
import pandas as pd
import csv

In [2]:
df_reviews = pd.read_json('./data/IMDB_reviews.json', lines=True).\
             drop_duplicates('review_text').sample(frac=1)
df_reviews.review_date = pd.to_datetime(df_reviews.review_date, infer_datetime_format=True)
df_reviews.user_id = df_reviews.user_id.astype('category')
print(df_reviews.shape)
df_reviews.tail()

(573385, 7)


,is_spoiler,movie_id,rating,review_date,review_summary,review_text,user_id
401955,True,tt0472043,7,2006-12-07,Spectacle vs. Story -- and story loses,"From technical and aesthetic viewpoints, this ...",ur2247078
296507,False,tt0227538,8,2001-06-11,Great!,Spy Kids is a wonderful kids movie. I give it...,ur1215753
159277,False,tt0903624,3,2013-01-03,An abject disaster. An unmitigated mess of a ...,It's hard to know where to begin when reviewin...,ur22144230
554224,True,tt0122690,7,2008-10-17,Ronin,From director John Frankenheimer (Birdman of A...,ur4248714
273620,False,tt0208092,10,2002-01-26,better than pulp fiction,This film is unquestionably one of the best fi...,ur1101005


In [3]:
# only less than 20% of the user comments' explicitly say that they contain spoilers
df_reviews.review_text.str.contains('spoiler', case=False).sum()/df_reviews.is_spoiler.sum()

0.17516041788195366

In [4]:
# imdb information about the movies
df_movies = pd.read_json('./data/IMDB_movie_details.json', lines=True)
df_movies.release_date = pd.to_datetime(df_movies.release_date, infer_datetime_format=True)
print(df_movies.shape)
df_movies.tail()

(1572, 7)


,duration,genre,movie_id,plot_summary,plot_synopsis,rating,release_date
1567,1h 53min,"[Sci-Fi, Thriller]",tt0289879,Evan Treborn grows up in a small town with his...,"In the year 1998, Evan Treborn (Ashton Kutcher...",7.7,2004-01-23
1568,1h 41min,[Drama],tt1723811,Brandon is a 30-something man living in New Yo...,"Brandon (Michael Fassbender) is a successful, ...",7.2,2012-01-13
1569,1h 46min,"[Action, Drama, History]",tt5013056,Evacuation of Allied soldiers from the British...,The film alternates between three different pe...,8.1,2017-07-21
1570,1h 33min,"[Comedy, Drama]",tt0104014/,"For a while now, beautiful 24-year-old Diana B...",,5.3,1992-02-21
1571,1h 32min,"[Drama, Thriller]",tt0114142/,"The marriage of David Burgess, a senior execut...",,4.0,1999-01-29


In [5]:
# join the two dataframe according to movie id
df_reviews = df_reviews.merge(df_movies, on="movie_id",how="left",suffixes=('_review','_movie'))

In [6]:
columns_keep = ['is_spoiler', 'movie_id', 'plot_summary', 'review_summary', 'review_text',]
all_columns = [review for review in df_reviews]
for x in columns_keep:
    all_columns.remove(x)
drop_columns = all_columns
df_train = df_reviews.drop(columns=drop_columns)
df_train['review_text'] = df_reviews['review_summary']+ ' ' + df_reviews['review_text']
df_train = df_train.drop(columns=['review_summary'])
df_train.tail()

,is_spoiler,movie_id,review_text,plot_summary
573380,True,tt0472043,Spectacle vs. Story -- and story loses From te...,"In the Maya civilization, a peaceful tribe is ..."
573381,False,tt0227538,Great! Spy Kids is a wonderful kids movie. I ...,Gregorio and Ingrid are the two greatest secre...
573382,False,tt0903624,An abject disaster. An unmitigated mess of a ...,Bilbo Baggins is swept into a quest to reclaim...
573383,True,tt0122690,Ronin From director John Frankenheimer (Birdma...,Ronin is the Japanese word used for Samurai wi...
573384,False,tt0208092,better than pulp fiction This film is unquesti...,Turkish and his close friend/accomplice Tommy ...


In [8]:
df_train.to_csv('./data/train.csv', index=False)